### check
   * exchange balance of bitocin v.s. stables
   * large transfer in of stables v.s. bitcoins -> large trading volume? -> cause price move?

In [1]:
from cdcqr.data.glassnode import glassnode_data as gnd
from cdcqr.ct.utils import plot2
import pandas as pd
import cdcqr.analytics.utils as autil 
from importlib import reload
import seaborn as sns
import matplotlib.pyplot as plt
from cdcqr.analytics.utils import calculate_corr_pvalues
from cdcqr.common.utils import LOCAL_DATA_DIR
import os
import numpy as np
from cdcqr.backtest.vbt.runsignal import runsignal
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)
import time
from IPython.display import Audio
sound_file = '/core/tmp/ding.wav'
%matplotlib inline

from cdcqr.common.utils import parallel_jobs

C:\Users\Wang\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
runsignal?

In [3]:
gn = gnd.GlassnodeData()

In [4]:
gn.get_feature_best_resolutions('balance_exchanges')

'10m'

In [5]:
[x for x in gn.get_feature_assets('balance_exchanges') if 'USD' in x]

['GUSD', 'USDT', 'USDP', 'USDC', 'BUSD', 'USDK', 'sUSD', 'HUSD']

In [6]:
reload(gnd)
gn = gnd.GlassnodeData()

In [7]:
fs = ['price_usd_close', 'balance_exchanges']
assets = ['BTC','USDT', 'USDC', 'BUSD']
df = gn.load_features(fs, assets, resolution='10m')

loading market/price_usd_close asset=BTC, resolution=10m
loading market/price_usd_close asset=USDT, resolution=10m
loading market/price_usd_close asset=USDC, resolution=10m
loading market/price_usd_close asset=BUSD, resolution=10m
loading distribution/balance_exchanges asset=BTC, resolution=10m
loading distribution/balance_exchanges asset=USDT, resolution=10m
loading distribution/balance_exchanges asset=USDC, resolution=10m
loading distribution/balance_exchanges asset=BUSD, resolution=10m


In [ ]:
df[df.index>='20200101'][['USDT-balance_exchanges','USDC-balance_exchanges','BUSD-balance_exchanges']].plot.area()

In [ ]:
df1 = df[df.index>='20200101'].ffill().drop(['USDT-price_usd_close','USDC-price_usd_close','BUSD-price_usd_close'],axis=1)
df1['stable-balance_exchanges'] = df['USDT-balance_exchanges']+df['USDC-balance_exchanges']+df['BUSD-balance_exchanges']
df1 = df1[['BTC-price_usd_close', 'stable-balance_exchanges','BTC-balance_exchanges']]
df1.head()

In [ ]:
df1['BTC-balance_exchanges_MC'] = df1['BTC-price_usd_close'] * df1['BTC-balance_exchanges']

In [ ]:
df1.plot2('BTC-balance_exchanges','stable-balance_exchanges')

In [ ]:
df1.plot2('BTC-balance_exchanges_MC','stable-balance_exchanges')

In [ ]:
df1['BTCSSR'] = df1['BTC-balance_exchanges_MC']/df1['stable-balance_exchanges']
df2 = df1.drop('BTC-balance_exchanges_MC', axis=1).dropna()

In [ ]:
fig, ax = plt.subplots()
sns.heatmap(df2.corr(method='pearson'), annot=True, fmt='.4f', 
            cmap=plt.get_cmap('coolwarm'), cbar=False, ax=ax)

In [ ]:
df3 = df2.copy()
df3['ret'] = df3['BTC-price_usd_close'].pct_change()
df3['r+5'] = df3['BTC-price_usd_close'].pct_change(periods=5).shift(-5)
df3['r+10'] = df3['BTC-price_usd_close'].pct_change(periods=10).shift(-10)
df3['r+100'] = df3['BTC-price_usd_close'].pct_change(periods=100).shift(-100)

df3['stable_bal_chg'] = df3['stable-balance_exchanges'].diff()
df3['stable_bal_chg_lag1'] = df3['stable_bal_chg'].shift(1)
df3['BTCSSR_lag1'] = df3['BTCSSR'].shift(1)
df3['BTCSSR_chg'] = df3['BTCSSR'].diff()
df3['BTCSSR_chg_lag1'] = df3['BTCSSR_chg'].shift(1)

In [ ]:
df3.dropna().count()

In [ ]:
df4 = df3.dropna()[['stable_bal_chg', 'stable_bal_chg_lag1', 'BTCSSR', 'BTCSSR_chg', 'BTCSSR_chg_lag1', 'ret', 'r+5', 'r+10', 'r+100']]

In [ ]:
df4.count()

In [ ]:
df4a = df4.where(~np.isinf(df4), 0)

In [ ]:
fig, ax = plt.subplots()
sns.heatmap(df4a.corr(method='pearson'), annot=True, fmt='.4f', 
            cmap=plt.get_cmap('coolwarm'), cbar=False, ax=ax)

In [ ]:
calculate_corr_pvalues(df4a)

### Backtest

In [ ]:
price = pd.read_pickle(os.path.join(LOCAL_DATA_DIR,'BTC-PERP@ftx.pickle'))

In [ ]:
df4a.groupby(pd.cut(df4a['BTCSSR_chg_lag1'], 10)).agg({'ret': lambda x: x.mean(skipna=True)})

In [ ]:
df4a.head(2)

In [ ]:
lookback = 4000 # ~= 1 months
df4a['BTCSSR_chg_lag1_mv4k'] = df4a['BTCSSR_chg_lag1'].rolling(lookback).mean()

In [ ]:
t0 = time.time()
pctrank = lambda x: pd.Series(x).rank(pct=True).iloc[-1]
df4a['BTCSSR_chg_lag1_pctrank4k'] = df4a['BTCSSR_chg_lag1'].rolling(lookback).apply(pctrank)
print(time.time()-t0)
Audio(sound_file, autoplay=True)

In [ ]:
t0 = time.time()
pctrank = lambda x: pd.Series(x).rank(pct=True).iloc[-1]
df4a['BTCSSR_pctrank4k'] = df4a['BTCSSR'].rolling(lookback).apply(pctrank)
print(time.time()-t0)
Audio(sound_file, autoplay=True)

In [ ]:
df4b = df4a[df4a.index>='20210101']

In [ ]:
df4b[['ret','r+5','r+10','r+100']].describe()

In [ ]:
df4b_gp = df4b.groupby(pd.cut(df4b['BTCSSR_chg_lag1_pctrank4k'], 50)).agg({'ret': lambda x: x.mean(skipna=True),
                                                                 'r+5': lambda x: x.mean(skipna=True), 
                                                                 'r+10': lambda x: x.mean(skipna=True),
                                                                 'r+100': lambda x: x.mean(skipna=True)})
df4b_gp.plot()

In [ ]:
df4b.groupby(pd.cut(df4b['BTCSSR_chg_lag1_pctrank4k'], 20)).agg({'ret': lambda x: x.mean(skipna=True),
                                                                 'r+5': lambda x: x.mean(skipna=True), 
                                                                 'r+10': lambda x: x.mean(skipna=True),
                                                                 'r+100': lambda x: x.mean(skipna=True)})

In [ ]:
df4b_gp2 = df4b.groupby(pd.cut(df4b['BTCSSR_pctrank4k'],50)).agg({'ret': lambda x: x.mean(skipna=True),
                                                                 'r+5': lambda x: x.mean(skipna=True), 
                                                                 'r+10': lambda x: x.mean(skipna=True),
                                                                 'r+100': lambda x: x.mean(skipna=True)})
df4b_gp2.plot()

In [ ]:
freq='1Min'
sl=4
tp=5
p=price.resample('1Min').last().ffill()
request = ['pf','backtest']

In [ ]:
t0 = time.time()
signalid = 'long-BTCSSR_pctrank4k'
feature = 'BTCSSR_pctrank4k'
short=False

In [ ]:
res=[]
price = p.copy()
counter0 = 0
for qtl in [0.995, 0.99, 0.975, 0.96]:
    signal = df4b[feature]>qtl # rolling max, keep the singal for a period (1 hour), good for combining with other signals
    signal = signal.reindex(p.index).ffill()
    for tp in [3,5,7, 10, 15, 20]:
        for sl in [0.5, 1, 2, 5, 10, 15, 20]:
            counter0 = counter0 + 1
            print(f'{counter0} : tp:{tp} sl:{sl} id:{signalid}')
            for n1 in [1,10,100]: # ignore this first
                for n2 in [101, 200, 1000]: # 
                    ressig=runsignal(price,signal=signal,tp=tp,sl=sl,ts=sl,n1=n1,n2=n2,rsil=30,rsih=50,rsilag=14,maf=5,mas=15,short=short,size=np.inf,fees=0.0007,freq='1Min',init_cash=10000,request=['pf','backtest'])
                    for k in ressig:
                        resd={"signalid":signalid,"qtl":qtl,"short":short,"tp":tp,"sl":sl,'k':k,'sr':ressig[k].sharpe_ratio(),'n1':n1,'n2':n2,'tr':ressig[k].total_return(),'ntrades':ressig[k].trades.count()/46.5} # rescale to number of weeks
                        res.append(resd)
print(time.time()-t0)
Audio(sound_file, autoplay=True)

In [ ]:
signal.to_frame().save('sample_signal')

### convert into single loop

In [ ]:
qtl = 0.99
signal = df4b[feature]>qtl # rolling max, keep the singal for a period (1 hour), good for combining with other signals
signal = signal.reindex(p.index).ffill()

signal

In [ ]:
backtest_configs = []
for tp in [3, 5]:  
    for sl in [1, 2]:
        for n1 in [1,10]:
            for n2 in [101, 200]:
                param_dict = {}
                param_dict['tp'] = tp
                param_dict['sl'] = sl
                param_dict['n1'] = n1
                param_dict['n2'] = n2
                backtest_configs.append(param_dict)

In [ ]:
import json
json.dumps(backtest_configs[1])

In [ ]:
hash(tuple(sorted(backtest_configs[1].items())))

In [ ]:
def partial_runsignal(param_dict):
    tp = param_dict['tp']
    sl = param_dict['sl']
    n1 = param_dict['n1']
    n2 = param_dict['n2']
    ressig=runsignal(price,signal=signal,tp=tp,sl=sl,ts=sl,n1=n1,n2=n2,rsil=30,rsih=50,rsilag=14,
                 maf=5,mas=15,short=short,size=np.inf,fees=0.0007,freq='1Min',init_cash=10000,request=['pf','backtest'])
    for k in ressig:
        resd={"signalid":signalid,"qtl":qtl,"short":short,"tp":tp,"sl":sl,'k':k,
              'sr':ressig[k].sharpe_ratio(),'n1':n1,'n2':n2,'tr':ressig[k].total_return(),'ntrades':ressig[k].trades.count()/46.5} # rescale to number of weeks
    return resd

In [ ]:
from cdcqr.backtest.alfafactory import runsignal


In [ ]:
ret = parallel_jobs(partial_runsignal, backtest_configs)



In [ ]:
pd.DataFrame(backtest_configs[1].values()).save('rest')

In [ ]:
partial_runsignal

In [ ]:
t0 = time.time()
signalid = 'long-BTCSSR_pctrank4k'
feature = 'BTCSSR_pctrank4k'
short=False

res=[]
price = p.copy()
counter0 = 0
for qtl in [0.995, 0.99, 0.975, 0.96]:
    signal = df4b[feature]>qtl # rolling max, keep the singal for a period (1 hour), good for combining with other signals
    signal = signal.reindex(p.index).ffill()
    for tp in [3,5,7, 10, 15, 20]:
        for sl in [0.5, 1, 2, 5, 10, 15, 20]:
            counter0 = counter0 + 1
            print(f'{counter0} : tp:{tp} sl:{sl} id:{signalid}')
            for n1 in [1,10,100]: # ignore this first
                for n2 in [101, 200, 1000]: # 
                    ressig=runsignal(price,signal=signal,tp=tp,sl=sl,ts=sl,n1=n1,n2=n2,rsil=30,rsih=50,rsilag=14,maf=5,mas=15,short=short,size=np.inf,fees=0.0007,freq='1Min',init_cash=10000,request=['pf','backtest'])
                    for k in ressig:
                        resd={"signalid":signalid,"qtl":qtl,"short":short,"tp":tp,"sl":sl,'k':k,'sr':ressig[k].sharpe_ratio(),'n1':n1,'n2':n2,'tr':ressig[k].total_return(),'ntrades':ressig[k].trades.count()/46.5} # rescale to number of weeks
                        res.append(resd)
print(time.time()-t0)
Audio(sound_file, autoplay=True)

In [ ]:
tp=10
sl=10
n1=1
n2=1000
qtl=0.96
signal = df4b[feature]>qtl # rolling max, keep the singal for a period (1 hour), good for combining with other signals
signal = signal.reindex(p.index).ffill()

ressig=runsignal(price,signal=signal,tp=tp,sl=sl,ts=sl,n1=n1,n2=n2,rsil=30,rsih=50,rsilag=14,maf=5,mas=15,short=short,size=np.inf,fees=0.0007,freq='1Min',init_cash=10000,request=['pf','backtest'])

In [ ]:
ressig['n_n_pf'].plot(subplots=['drawdowns','trade_pnl','cum_returns'])

In [ ]:
t0 = time.time()
signalid = 'short-BTCSSR_chg_lag1'
feature = 'BTCSSR_chg_lag1_pctrank4k'
short=False

res=[]
price = p.copy()
counter0 = 0
for qtl in [0.99, 0.975, 0.95, 0.925]:
    signal = df4b[feature]>qtl
    signal = signal.reindex(p.index).ffill()
    for tp in [3,5,7]:
        for sl in [0.5, 1, 2]:
            counter0 = counter0 + 1
            print(f'{counter0} : tp:{tp} sl:{sl} id:{signalid}')
            for n1 in [1,10,100]: #
                for n2 in [101, 200, 1000]:
                    ressig=runsignal(price,signal=signal,tp=tp,sl=sl,ts=sl,n1=n1,n2=n2,rsil=30,rsih=50,rsilag=14,maf=5,mas=15,short=short,size=np.inf,fees=0.0007,freq='1Min',init_cash=10000,request=['pf','backtest'])
                    for k in ressig:
                        resd={"signalid":signalid,"qtl":qtl,"short":short,"tp":tp,"sl":sl,'k':k,'sr':ressig[k].sharpe_ratio(),'n1':n1,'n2':n2,'tr':ressig[k].total_return(),'ntrades':ressig[k].trades.count()/46.5} # rescale to number of weeks
                        res.append(resd)
print(time.time()-t0)
Audio(sound_file, autoplay=True)

In [ ]:
print(time.time()-t0)

In [ ]:
res_df = pd.DataFrame(res)

In [ ]:
res_df['short'].value_counts()

In [ ]:
res_df.save('long-BTCSSR_pctrank4k_res_df')

In [ ]:
res_df.pv()

In [ ]:
res_df.pv()

res_df.to_pickle()

## obv 
    * extend range for tp and sl
    * larger set of n1 and n2 seems good?

In [ ]:
dfr = pd.DataFrame(res)

In [ ]:
# dfr.to_pickle(os.path.join(LOCAL_DATA_DIR, 'ssr_bt_res_1.pkl'))

In [ ]:
dfr.shape

In [ ]:
dfr.pv()

In [ ]:
dfr['ma_tpsl_pf'].plot(subplots=['drawdowns','trade_pnl','cum_returns'])

In [ ]:
# select subset and check
tp = 3
sl = 1
ts = 1
nn = 100
signal = getsignal3(0.99,20, 0.0,0)['s']
price = price0
short=True
ressig=runsignal(price,signal=signal,tp=tp,sl=sl,ts=ts,n1=1,n2=60,rsil=30,rsih=50,rsilag=14,maf=5,mas=15,short=short,size=np.inf,fees=0.0007,freq='1Min',init_cash=10000,request=['pf'])
ressig['ma_tpsl_pf'].plot(subplots=['drawdowns','trade_pnl','cum_returns'])

In [ ]:
t0 = time.time()
signalid = 'short-BTCSSR_pctrank4k'
feature = 'BTCSSR_pctrank4k'
short=True

res=[]
price = p.copy()
counter0 = 0
for qtl in [0.005, 0.01, 0.02, 0.03, 0.05]:
    signal = df4b[feature]<qtl # rolling max, keep the singal for a period (1 hour), good for combining with other signals
    signal = signal.reindex(p.index).ffill()
    for tp in [3,5,7, 10, 15, 20]:
        for sl in [0.5, 1, 2, 5, 10, 15, 20]:
            counter0 = counter0 + 1
            print(f'{counter0} : tp:{tp} sl:{sl} id:{signalid}')
            for n1 in [1,10,100]: # ignore this first
                for n2 in [101, 200, 1000]: # 
                    ressig=runsignal(price,signal=signal,tp=tp,sl=sl,ts=sl,n1=n1,n2=n2,rsil=30,rsih=50,rsilag=14,maf=5,mas=15,short=short,size=np.inf,fees=0.0007,freq='1Min',init_cash=10000,request=['pf','backtest'])
                    for k in ressig:
                        resd={"signalid":signalid,"qtl":qtl,"short":short,"tp":tp,"sl":sl,'k':k,'sr':ressig[k].sharpe_ratio(),'n1':n1,'n2':n2,'tr':ressig[k].total_return(),'ntrades':ressig[k].trades.count()/46.5} # rescale to number of weeks
                        res.append(resd)
print(time.time()-t0)
Audio(sound_file, autoplay=True)

In [ ]:
res_df_s = pd.DataFrame(res)

In [ ]:
res_df_s.save('short-BTCSSR_pctrank4k_res_df')

In [ ]:
res_df_s.pv()

In [ ]:
a = []
b = [1,2]

a.extend(b)
a.extend(b)